<a href="https://colab.research.google.com/github/khushbooekhande/DrugRecommendationSystem/blob/master/DrugRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd 
import numpy as np

In [0]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(f'/content/drive/My Drive/sentiment_review.csv')
df=df.drop(['Symptoms', 'Rating', 'Unnamed: 0'], axis=1)
df.Drug = df.Drug.str.upper()

In [0]:
predicted_disease = 'schizophrenia'

In [0]:
# function to get drugs based on sentiment review
def _DrugRecommender(df=df, predicted_disease=predicted_disease):
    #df['Rating'] = np.where(df['Rating']>7, 1, 0)
    df = df.loc[df['Disease'] == predicted_disease]
    df = df.loc[df['sentiment_review'] == 'positive']
    df = df.sort_values(by = ['UsefulCount'], ascending=False)
    df = df[['Disease', 'Drug']]
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df.head()

    return df

In [0]:
drug = _DrugRecommender()

In [28]:
print('List of recommened drugs based on useful count and sentiment analysis')
drug

List of recommened drugs based on useful count and sentiment analysis


,Disease,Drug
0,schizophrenia,CLOZAPINE
1,schizophrenia,CLOZARIL
2,schizophrenia,RISPERDAL
3,schizophrenia,RISPERIDONE
4,schizophrenia,LURASIDONE


In [0]:
sd = pd.read_csv(f'/content/drive/My Drive/SideEffects_processed_new.csv')
sd = sd.drop(['condition', 'commentsReview'], axis=1)
sd = sd.dropna()

In [0]:
sd.sideEffects = sd.sideEffects.replace('Moderate Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Mild Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Severe Side Effects', 1)
sd.sideEffects = sd.sideEffects.replace('No Side Effects', 3)
sd.sideEffects = sd.sideEffects.replace('Extremely Severe Side Effects', 0)

In [0]:
sd.effectiveness = sd.effectiveness.replace('Considerably Effective', 2)
sd.effectiveness = sd.effectiveness.replace('Highly Effective', 3)
sd.effectiveness = sd.effectiveness.replace('Marginally Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Moderately Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Ineffective', 0)

In [0]:
sd = sd[['urlDrugName', 'rating', 'effectiveness', 'sideEffects', 'sideEffectsReview']]

In [0]:
sd = sd.rename(columns={"urlDrugName": "Drug", "rating": "Rating", "effectiveness": "Effectivness Rating",
                       "sideEffects": "Side Effect Rating", "sideEffectsReview": "Side Effects"})
sd.Drug = sd.Drug.str.upper()

In [34]:
print('Drug with possible side effects')
sd

Drug with possible side effects


,Drug,Rating,Effectivness Rating,Side Effect Rating,Side Effects
0,ENALAPRIL,4,3,2,"cough, hypotension , proteinuria, impotence , ..."
1,ORTHO-TRI-CYCLEN,1,3,1,"Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon..."
2,PONSTEL,10,3,3,Heavier bleeding and clotting than normal.
3,PRILOSEC,3,1,2,"Constipation, dry mouth and some mild dizzines..."
4,LYRICA,2,1,1,I felt extremely drugged and dopey. Could not...
...,...,...,...,...,...
4139,ACCUTANE,7,2,1,"Hairloss, extreme dry skin, itchiness, raises ..."
4140,PROAIR-HFA,10,3,3,I have experienced no side effects.
4141,ACCUTANE,8,2,2,"Painfull muscles, problems with seeing at night"
4142,DIVIGEL,10,3,3,No side effects of any kind were noted or appa...


In [0]:
def _probScore(df, w1, w2, w3):
    return(1-(df[w1] * df[w2] * df[w3]).sum()/df[w1].sum()/10)

In [0]:
def _getSE(df, sedf, l):
    sdf = sedf
    sdf = sdf.loc[sdf['Drug'] == l]
    #print(sdf)
    w = sdf.groupby(["Drug"]).apply(_probScore, "Rating", "Effectivness Rating", "Side Effect Rating")
    sdf1 = sdf.loc[sdf['Effectivness Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Side Effect Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Rating'] <= 1 ]
    sdf1 = sdf1.reset_index()
    sdf1 = pd.DataFrame(sdf['Side Effects'])
    df1 = pd.DataFrame(w, columns=['Prob. of Side Effect'])
    df1 = df1.reset_index()
    #print(sdf[:1]['Side Effects'])
    df2 = pd.DataFrame(sdf['Side Effects'])
    df2 = df2.reset_index()
    df2 = df2.drop(['index'], axis=1)
    dd = pd.concat([df1, df2], axis=1)
    
    return dd

In [0]:
def _getSideEffects(df, sedf):
    l = list(df.Drug)
    #print(l)
    for i in range(0, 1):
        df1 = _getSE(df, sedf, l[i])
    dd = df1
    for i in range(1, len(l)):
        df2 = _getSE(df, sedf, l[i])
        dd = pd.concat([dd, df2])
    return dd

In [0]:
se = _getSideEffects(drug, sd)

In [41]:
recommender = drug.set_index('Drug').join(se.set_index('Drug'))
recommender = recommender.reset_index()
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna(0)
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna('Not Available')
print('Mapped recommended drugs with possible side effects and probabilistic score')
recommender

Mapped recommended drugs with possible side effects and probabilistic score


,Drug,Disease,Prob. of Side Effect,Side Effects
0,CLOZAPINE,schizophrenia,0.100000,"Blurred vision, confusion, dizziness, fever, s..."
1,CLOZARIL,schizophrenia,0.600000,"Blurred vision, trembling of hands or feet, c..."
2,LURASIDONE,schizophrenia,0.800000,"Absence of or decrease in body movement, muscl..."
3,RISPERDAL,schizophrenia,0.779167,"I suffered increased anxiety, more dilusions, ..."
4,RISPERIDONE,schizophrenia,0.800000,"Anxiety, agitation, inability to move eyes, ma..."


In [43]:
#sorting based on probabilistic score
recommender = recommender.sort_values('Prob. of Side Effect')
print("The recommended Drugs for the given Disease is:")
recommender

The recommended Drugs for the given Disease is:


,Drug,Disease,Prob. of Side Effect,Side Effects
0,CLOZAPINE,schizophrenia,0.100000,"Blurred vision, confusion, dizziness, fever, s..."
1,CLOZARIL,schizophrenia,0.600000,"Blurred vision, trembling of hands or feet, c..."
3,RISPERDAL,schizophrenia,0.779167,"I suffered increased anxiety, more dilusions, ..."
2,LURASIDONE,schizophrenia,0.800000,"Absence of or decrease in body movement, muscl..."
4,RISPERIDONE,schizophrenia,0.800000,"Anxiety, agitation, inability to move eyes, ma..."
